In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from zipfile import ZipFile
import cartopy
import cartopy.crs as ccrs
import geopandas as gpd
import shapely

In [3]:
sb_inat = pd.read_csv("/courses/EDS214/EDS220_gbffs/inat_sb_recent.csv", sep = '\t') #make sure to update the file path to where your data is stored

In [4]:
tarantulas = sb_inat.loc[sb_inat['family'] == "Theraphosidae"]

In [34]:
list(tarantulas.columns)

['gbifID',
 'datasetKey',
 'occurrenceID',
 'kingdom',
 'phylum',
 'class',
 'order',
 'family',
 'genus',
 'species',
 'infraspecificEpithet',
 'taxonRank',
 'scientificName',
 'verbatimScientificName',
 'verbatimScientificNameAuthorship',
 'countryCode',
 'locality',
 'stateProvince',
 'occurrenceStatus',
 'individualCount',
 'publishingOrgKey',
 'decimalLatitude',
 'decimalLongitude',
 'coordinateUncertaintyInMeters',
 'coordinatePrecision',
 'elevation',
 'elevationAccuracy',
 'depth',
 'depthAccuracy',
 'eventDate',
 'day',
 'month',
 'year',
 'taxonKey',
 'speciesKey',
 'basisOfRecord',
 'institutionCode',
 'collectionCode',
 'catalogNumber',
 'recordNumber',
 'identifiedBy',
 'dateIdentified',
 'license',
 'rightsHolder',
 'recordedBy',
 'typeStatus',
 'establishmentMeans',
 'lastInterpreted',
 'mediaType',
 'issue',
 'geometry']

In [35]:
tarantulas.dtypes

gbifID                                 int64
datasetKey                            object
occurrenceID                          object
kingdom                               object
phylum                                object
class                                 object
order                                 object
family                                object
genus                                 object
species                               object
infraspecificEpithet                  object
taxonRank                             object
scientificName                        object
verbatimScientificName                object
verbatimScientificNameAuthorship     float64
countryCode                           object
locality                             float64
stateProvince                         object
occurrenceStatus                      object
individualCount                      float64
publishingOrgKey                      object
decimalLatitude                      float64
decimalLon

In [6]:
# Create geometries from your lat-lons
geom_list = [shapely.geometry.Point(lon,lat) for lon,lat in zip(tarantulas["decimalLongitude"] , tarantulas["decimalLatitude"])] # check the ordering of lon/lat

# create geopandas geodataframe
tarantulas_gdf = gpd.GeoDataFrame(tarantulas, geometry=geom_list, crs={"init":"EPSG:4326"})

/opt/python/3.10.5/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [7]:
tarantulas_gdf.crs # it's in degrees, we want it in meters! we should reproject it

<Geographic 2D CRS: +init=epsg:4326 +type=crs>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
tarantulas_rpj = tarantulas_gdf.to_crs("EPSG:6423") # NAD83 California Zone 5-- includes SB

tarantulas_rpj.crs # now the unit of measurement is in meters! 

<Derived Projected CRS: EPSG:6423>
Name: NAD83(2011) / California zone 5
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - California - counties Kern; Los Angeles; San Bernardino; San Luis Obispo; Santa Barbara; Ventura.
- bounds: (-121.42, 32.76, -114.12, 35.81)
Coordinate Operation:
- name: SPCS83 California zone 5 (meters)
- method: Lambert Conic Conformal (2SP)
Datum: NAD83 (National Spatial Reference System 2011)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [25]:
ucsb_df = pd.DataFrame(
    {'Location': ['Bren Hall'],
     'Latitude': [34.413059],
     'Longitude': [-119.841826]})

ucsb_gdf = gpd.GeoDataFrame(
    ucsb_df, geometry=gpd.points_from_xy(ucsb_df.Longitude, ucsb_df.Latitude))

ucsb_gdf = ucsb_gdf.set_crs('EPSG:6423')

ucsb_buffer = ucsb_gdf.buffer(1610)

friendly_neighbors = tarantulas_rpj["geometry"].intersection(ucsb_buffer)

/opt/python/3.10.5/lib/python3.10/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


In [32]:
nearby_tarantulas = tarantulas_rpj[~friendly_neighbors.is_empty]

len(nearby_tarantulas) # 53 tarantulas have been reported within 1 mile of UCSB since 2008

nearby_tarantulas

/opt/python/3.10.5/lib/python3.10/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


gbifID                            datasetKey  \
2201    1898831738  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
7013    2563466165  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
10105   2294451477  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
10215   2423071640  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
11666   2826483417  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
13170   2445037736  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
13965   2563472519  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
15564   2898321022  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
17353   2350387486  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
18116   3335121543  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
21552   2868187641  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
23709   2350389502  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
30982   2963857931  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
32114   3747182896  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
47626   3108985366  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
51240   2573835426  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
55444   2423254234  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
59498   2850936096  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
62806   2429540641  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
67771   2563434779  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
68075   1993774683  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
68931   2422915116  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
69564   2563658942  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
73228   3902868754  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
73589   2883102796  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
76180   2283010401  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
77424   2429340235  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
87959   1899798171  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
88021   3759863447  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
91075   1677314967  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
91482   2901612318  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
92300   1932339387  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
93191   2445121716  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
95176   2451701712  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
97508   2850576684  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
101186  1899775478  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
101272  3399221468  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
101719  2423253888  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
103400  2563428938  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
107116  1677301127  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
111392  2563433464  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
117538  2444991867  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
119158  2850897341  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
122706  2422831086  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
125532  1702474716  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
125628  2898418351  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
126228  1677293239  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
129818  3925512518  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
134137  2826522939  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
135522  2851011537  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
135711  2641413418  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
137610  2868198792  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
141418  2563434890  50c9509d-22c7-4a22-a47d-8c48425ef4a7   

                                             occurrenceID   kingdom  \
2201    https://www.inaturalist.org/observations/15782961  Animalia   
7013    https://www.inaturalist.org/observations/33991504  Animalia   
10105   https://www.inaturalist.org/observations/28654164  Animalia   
10215   https://www.inaturalist.org/observations/32731888  Animalia   
11666   https://www.inaturalist.org/observations/54586761  Animalia   
13170   https://www.inaturalist.org/observations/34541639  Animalia   
13965   https://www.inaturalist.org/observations/35937770  Animalia   
15564   https://www.inaturalist.org/observations/62418607  Animalia   
17353   https://www.inaturalist.org/observations/30655276  Animalia   
18116   https://www.inaturalist.org/observations/88076440  An

In [ ]:

ucsb_df = pd.DataFrame(
    {'Location': ['Bren Hall'],
     'Latitude': [34.413059],
     'Longitude': [-119.841826]})

ucsb_gdf = gpd.GeoDataFrame(
    ucsb_df, geometry=gpd.points_from_xy(ucsb_df.Longitude, ucsb_df.Latitude), crs={"init":"EPSG:4326"})

ucsb_gdf = ucsb_gdf.to_crs("EPSG:6423")

ucsb_buffer = ucsb_gdf.buffer(16090).unary_union

friendly_neighbors = tarantulas_rpj["geometry"].within(ucsb_buffer)

friendly_neighbors